In [0]:
import numpy as np
import pandas as pd
import os
import io
from sklearn.preprocessing import LabelEncoder
import time
import copy

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!cp -r /content/gdrive/My\ Drive/MURA_files/data/MURA-v1.1.zip MURA-v1.1.zip
!unzip -q MURA-v1.1.zip

In [0]:
!cp -r /content/gdrive/My\ Drive/MURA_files/data/processed processed

In [0]:
!ls /content/processed

train_all.csv	   train_hand.csv      valid_all.csv	  valid_hand.csv
train_elbow.csv    train_humerus.csv   valid_elbow.csv	  valid_humerus.csv
train_finger.csv   train_shoulder.csv  valid_finger.csv   valid_shoulder.csv
train_forearm.csv  train_wrist.csv     valid_forearm.csv  valid_wrist.csv


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [0]:
mura_prefix = '/content/'
train_all = '/content/processed/train_all.csv'
valid_all = '/content/processed/valid_all.csv'

In [0]:
class MuraDataset(data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None, columns=[3, 4]):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            columns (int array, optional): an array of length 2. first element
                is the column that specifies path in the csv and second is
                the label for that path
        """
        self.process_csv(csv_file, columns)
        self.root_dir = root_dir
        self.transform = transform
        
    def process_csv(self, file, columns):
        # read and update
        df = pd.read_csv(file)
        df = df.iloc[:, columns]
        df.columns = ['path', 'label']
        
        # encode labels
        le = LabelEncoder()
        df.label = le.fit_transform(df.label)
        
        # save
        self.le = le
        self.mura_frame = df

    def __len__(self):
        return len(self.mura_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.mura_frame.iloc[idx, 0])
        
        image = cv2.imread(img_name, 0)
        ## thresholding
        #image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)  
        image = Image.fromarray(image)
        image = image.convert('L')
        image = self.transform(image)         
        
        label = self.mura_frame.iloc[idx, 1]
        #label = torch.from_numpy(label.reshape(1, 1))
        
        sample = {'image': image, 'label': label}

        return sample

In [0]:
trainSet = MuraDataset(csv_file=train_all, root_dir=mura_prefix, 
                       transform=transforms.Compose([transforms.Resize((227,227)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

devSet = MuraDataset(csv_file=valid_all, root_dir=mura_prefix, 
                       transform=transforms.Compose([transforms.Resize((227,227)),
                                                          transforms.ToTensor(),
                                                          transforms.Normalize(
                                                              mean=[0.456],
                                                              std= [0.225])]))

In [0]:
batch_size = 1024
trainLoader = data.DataLoader(trainSet, batch_size=batch_size, shuffle=True, num_workers=4)
devLoader = data.DataLoader(devSet, batch_size=batch_size, shuffle=True, num_workers=4)


dataDict = {'train': trainLoader, 'valid': devLoader}

In [0]:
def init_alexnet():
  alexnet = models.alexnet(pretrained=False)
  for param in alexnet.parameters():
    param.requires_grad = False
  alexnet.features._modules['0'] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  alexnet.classifier._modules['6'] = nn.Linear(in_features=4096, out_features=7, bias=True)
  alexnet.to(device)  # transfer to gpu

  optimizer = optim.Adam(alexnet.parameters(), lr=0.0013, weight_decay=1.0201e-06, amsgrad=True)
  criterion = nn.CrossEntropyLoss()
  return alexnet, optimizer, criterion

In [0]:
def train_model(model, dataloaders, criterion, optimizer, 
                model_id, save_path = "/content/models/",
                num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    train_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, batch in enumerate(dataloaders[phase], 0):# inputs, labels in dataloaders[phase]:
              #if i < 4:
                inputs = batch['image'].to(device)
                labels = batch['label'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                #running_accuracy = running_corrects.double() / ((i+1) * batch_size)
                
                #print('\tbatch: {} Loss: {:.4f} Acc: {:.4f}'.format(i + 1, running_loss, running_accuracy))
              
              #else: break

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            # append losses
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
            else:
                train_acc_history.append(epoch_acc)
                
            # save model
            save_model_path = save_path + model_id
            if not os.path.exists(save_model_path):
              os.makedirs(save_model_path)
            torch.save(model.state_dict(), save_model_path + "/epoch_" + str(epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

In [48]:
alexnet, optimizer, criterion = init_alexnet()
best, vh, th = train_model(alexnet, dataDict, criterion, optimizer, model_id = "alex_scratch", num_epochs=15)

Epoch 0/14
----------
train Loss: 1.5014 Acc: 0.4375
valid Loss: 1.3817 Acc: 0.4989

Epoch 1/14
----------
train Loss: 1.1309 Acc: 0.5971
valid Loss: 1.1260 Acc: 0.6143

Epoch 2/14
----------
train Loss: 0.9716 Acc: 0.6601
valid Loss: 0.9690 Acc: 0.6656

Epoch 3/14
----------
train Loss: 0.8752 Acc: 0.6963
valid Loss: 0.8476 Acc: 0.7129

Epoch 4/14
----------
train Loss: 0.8015 Acc: 0.7222
valid Loss: 0.8136 Acc: 0.7269

Epoch 5/14
----------
train Loss: 0.7535 Acc: 0.7363
valid Loss: 0.7574 Acc: 0.7491

Epoch 6/14
----------
train Loss: 0.7130 Acc: 0.7519
valid Loss: 0.7218 Acc: 0.7591

Epoch 7/14
----------
train Loss: 0.6835 Acc: 0.7634
valid Loss: 0.7113 Acc: 0.7598

Epoch 8/14
----------
train Loss: 0.6698 Acc: 0.7701
valid Loss: 0.6941 Acc: 0.7720

Epoch 9/14
----------
train Loss: 0.6362 Acc: 0.7808
valid Loss: 0.6530 Acc: 0.7782

Epoch 10/14
----------
train Loss: 0.6231 Acc: 0.7855
valid Loss: 0.6291 Acc: 0.7914

Epoch 11/14
----------
train Loss: 0.6042 Acc: 0.7911
valid Loss

alex net with thresholding -- best epoch 13 /content/models/alex_1

In [0]:
## copy models back to drive!!
!cp -r /content/models/alex_scratch /content/gdrive/My\ Drive/MURA_files/models/models/

In [53]:
!ls /content/gdrive/My\ Drive/MURA_files/models/models/

alex_1	alex_nothreshold  alex_scratch


In [0]:
accuracy_logs = pd.DataFrame({'epoch': np.arange(1, 16), 'train': [x.item() for x in th], 'valid': [x.item() for x in th]})
accuracy_logs.to_csv('accuracy_logs.csv')

In [0]:
!cp accuracy_logs.csv /content/gdrive/My\ Drive/MURA_files/models/models/alex_scratch/